In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "4"

import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import corner
import batman
import emcee
import time
from multiprocessing import Pool
from IPython.display import display, Math
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import matplotlib.ticker as tck

from exotic_ld import StellarLimbDarkening

import pandas as pd
import pickle

import celerite2
from celerite2 import terms

In [ ]:
def set_params(params, t, guess_flux, yerrs):
    """
    Creating a GaussianProcess object for a Matern32Term kernel. 

    Parameters
    ----------
    params : array_like
        Containing Matern32Term kernel parameters.
        _sigma, _rho = params 
    
    guess_flux : array_like
        Containing the physical (batman) model flux.
    
    yerrs : array_like
        Containing an array of flux errors from TESS observations.

    Returns
    -------
    gp : GaussianProcess
        GaussianProcess object.

    """
    _sigma, _rho = params
    kernel = terms.Matern32Term(sigma = _sigma, rho  = _rho)
    gp = celerite2.GaussianProcess(kernel, mean=guess_flux)
    gp.compute(t, yerr=yerrs, quiet=True)

    return gp

In [ ]:
TIC = 'TIC 292152376' #HAT-P-32b / TIC 292152376

search_result = lk.search_lightcurve(
    target=TIC,
    mission='TESS', author='SPOC')
print("trying to download")
lc_collection = search_result.download_all()
print("Search done")
print(lc_collection)
lc = lc_collection.stitch(corrector_func=lambda x: x.
                          remove_nans().normalize(unit='unscaled'))

lc = lc.remove_nans()

In [ ]:
lc

In [ ]:
#Removing an outlier

t = []
flux_data = []
yerrs = []

t = np.array(t)
flux_data = np.array(flux_data)
yerrs = np.array(yerrs)

In [ ]:
fig, ax = plt.subplots(1, figsize=(16,6))
plt.errorbar(t, flux_data, yerrs, fmt='.', color='Black', ecolor='LightGrey')


plt.ylabel("Normalised flux", size=17)
plt.xlabel("Time - 2457000 (BTJD days)", size=17)

ax.xaxis.set_tick_params(labelsize=17)
ax.yaxis.set_tick_params(labelsize=17)

ax.xaxis.set_minor_locator(tck.AutoMinorLocator())
ax.yaxis.set_minor_locator(tck.AutoMinorLocator())

plt.show()

In [ ]:
filename = "HAT_P_32b_free_ecc_gp_batman_flat_samples.p"
with open(filename, "rb") as f:
    flat_samples = pickle.load(f)
f.close()

In [ ]:
#batman model
model_T0 = np.percentile(flat_samples[:, 0], 50) #Mid-transit time (Days)
model_Rp = np.percentile(flat_samples[:, 1], 50) #Rp/Rs
model_per = np.percentile(flat_samples[:, 2], 50) #Orbital period (Days)
model_inc = np.percentile(flat_samples[:, 3], 50) #Orbital inclination (Deg)
model_a = np.percentile(flat_samples[:, 4], 50) #a/R*
model_ecc = np.percentile(flat_samples[:, 5], 50) #Eccentricity

#GP model
model_sigma = np.percentile(flat_samples[:, 6], 50)
model_rho = np.percentile(flat_samples[:, 7], 50)

prior_Rs = 1.219 #R_sun Hartman et al. 2011

model_params = batman.TransitParams()
model_params.t0 = model_T0
model_params.per = model_per
model_params.rp = model_Rp
model_params.a = model_a
model_params.inc = model_inc
model_params.ecc = model_ecc
model_params.w = 50.0 #(Deg) Wang et al. 2019
model_params.limb_dark = "quadratic"
model_params.u = [0.2366419727326458, 0.2819412888558852] #Estimated by HAT_P_32b.ipynb

m = batman.TransitModel(model_params, t) #Use TESS time series, t
batman_model = m.light_curve(model_params)

In [ ]:
gp = set_params([model_sigma, model_rho], t, batman_model, yerrs)

filename = "HAT_P_32b_model_flux_sigma.p"

if os.path.exists(filename) == True:
    with open(filename, "rb") as f:
        model_flux, variance = pickle.load(f)
    f.close()
else:
    model_flux, variance = gp.predict(flux_data, t=t, return_var=True) #model flux = gp model + batman model
    sigma = np.sqrt(variance)

    file = open(filename, "wb")
    pickle.dump([model_flux, sigma], file)
    file.close()

In [ ]:
#batman + gp model

fig, ax = plt.subplots(1, figsize=(16,6))

ax.errorbar(t, flux_data, yerr=yerrs,fmt='.', markersize=5, color='black', ecolor='LightGrey', label="Data", zorder=0) #Black
ax.plot(t, model_flux, color='Red', zorder=10)


# ax.set_xlim([min(t) - 1, max(t) + 1])
ax.set_ylim([0.967, 1.015])
ax.set_ylabel("Normalised flux", size=17)
ax.set_xlabel("Time - 2457000 (BTJD days)", size=17)

# Mirror y
axA = ax.twinx()
axA.set_ylim(ax.get_ylim())
axA.get_yaxis().set_ticklabels([])
axA.tick_params(which="both", direction="in")#, labelright=True, labelleft=True)

# Mirror x
axB = ax.twiny()
axB.set_xlim(ax.get_xlim())
axB.get_xaxis().set_ticklabels([])
axB.tick_params(which="both", direction="in")

ax.xaxis.set_tick_params(labelsize=17)
ax.yaxis.set_tick_params(labelsize=17)

#Enabling Minor ticks
ax.xaxis.set_minor_locator(tck.AutoMinorLocator())
axB.xaxis.set_minor_locator(tck.AutoMinorLocator())
ax.yaxis.set_minor_locator(tck.AutoMinorLocator())
axA.yaxis.set_minor_locator(tck.AutoMinorLocator())

plt.show()

In [ ]:
#Detrending the flux data
gp_model = model_flux - batman_model
detrended_flux_data = flux_data - gp_model

In [ ]:
t_range = np.arange(min(t), max(t), 0.001388889) #0.001388889 days = 2 minutes

t_range_m = batman.TransitModel(model_params, t_range) #Use TESS time series, t
batman_model_extend_range = t_range_m.light_curve(model_params)

In [ ]:
#detrended flux

fig, ax = plt.subplots(1, figsize=(16,6))

ax.errorbar(t, detrended_flux_data, yerr=yerrs,fmt='.', markersize=5, color='black', ecolor='LightGrey', label="Data", zorder=0) #Black
ax.plot(t_range, batman_model_extend_range, color='Red', zorder=10)


# ax.set_xlim([min(t), max(t)])
ax.set_ylim([0.986, 1.01])
ax.set_ylabel("Normalised flux", size=17)
ax.set_xlabel("Time - 2457000 (BTJD days)", size=17)

# Mirror y
axA = ax.twinx()
axA.set_ylim(ax.get_ylim())
axA.get_yaxis().set_ticklabels([])
axA.tick_params(which="both", direction="in")#, labelright=True, labelleft=True)

# Mirror x
axB = ax.twiny()
axB.set_xlim(ax.get_xlim())
axB.get_xaxis().set_ticklabels([])
axB.tick_params(which="both", direction="in")

ax.xaxis.set_tick_params(labelsize=17)
ax.yaxis.set_tick_params(labelsize=17)

#Enabling Minor ticks
ax.xaxis.set_minor_locator(tck.AutoMinorLocator())
axB.xaxis.set_minor_locator(tck.AutoMinorLocator())
ax.yaxis.set_minor_locator(tck.AutoMinorLocator())
axA.yaxis.set_minor_locator(tck.AutoMinorLocator())

plt.show()

In [ ]:
gp_batman_residuals = detrended_flux_data - model_flux

#Detrended flux data folding
lc_folded = lk.LightCurve(time=t, flux=detrended_flux_data, flux_err=yerrs).fold(model_per, model_T0)
t_folded = np.array(lc_folded.time.value)
detrended_flux_folded = np.array(lc_folded.flux)
yerrs_folded = np.array(lc_folded.flux_err)

#Phase folding model 
folded_model = lk.LightCurve(time=t, flux=batman_model).fold(model_per, model_T0)
folded_model_flux = np.array(folded_model.flux)
folded_model_time = np.array(folded_model.time.value)

#Phase folding residuals
folded_residuals = lk.LightCurve(time=t, flux=gp_batman_residuals).fold(model_per, model_T0)
folded_residuals_flux = np.array(folded_residuals.flux)
folded_residuals_time = np.array(folded_residuals.time.value)


In [ ]:
#Phased folded

#Panel settings
left = 0.1
bottom = 0.1
width = 0.65
height = 0.9
spacing = -1.15

rect = [left, bottom, width, height]
rect_res = [left, bottom+height+spacing, width, 0.15]

fig = plt.figure(figsize=(16, 6), dpi=500)
ax = fig.add_axes(rect)

ax_res = fig.add_axes(rect_res, sharex=ax)

#Top panel
ax.errorbar(t_folded, detrended_flux_folded, yerr=yerrs_folded,fmt='.', markersize=5, color='black', ecolor='LightGrey', label="Data", zorder=0)
ax.plot(folded_model_time, folded_model_flux, color='red',label="Model", lw=2, zorder=10)

ax.set(ylabel="Normalised flux")
ax.set_ylim([0.986, 1.01])
ax.xaxis.label.set_size(17)
ax.yaxis.label.set_size(17)


# Mirror y
axy = ax.twinx()
axy.set_ylim(ax.get_ylim())
axy.get_yaxis().set_ticklabels([])
axy.tick_params(which="both", direction="in")#, labelright=True, labelleft=True)

# Mirror x
axx = ax.twiny()
axx.set_xlim(ax.get_xlim())
axx.get_xaxis().set_ticklabels([])
axx.tick_params(which="both", direction="in")

axx.xaxis.set_minor_locator(ticker.AutoMinorLocator())
axy.yaxis.set_minor_locator(ticker.AutoMinorLocator())
ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax.xaxis.set_tick_params(labelsize=17)
ax.yaxis.set_tick_params(labelsize=17)

#Bottom panel
ax_res.scatter(folded_residuals_time, folded_residuals_flux, color='black', s=5)
# ax_res.set_ylim((-max(abs(folded_residuals_flux))*1.2, max(abs(folded_residuals_flux)*1.2)))


ax_res.set(xlabel="Phase", ylabel="Residuals")
ax_res.xaxis.label.set_size(17)
ax_res.yaxis.label.set_size(17)

# Mirror y
ax_res_y = ax_res.twinx()
ax_res_y.set_ylim(ax_res.get_ylim())
ax_res_y.get_yaxis().set_ticklabels([])
ax_res_y.tick_params(which="both", direction="in")

# Mirror x
ax_res_x = ax_res.twiny()
ax_res_x.set_xlim(ax_res.get_xlim())
ax_res_x.get_xaxis().set_ticklabels([])
ax_res_x.tick_params(which="both", direction="in")

ax_res_x.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax_res_y.yaxis.set_minor_locator(ticker.AutoMinorLocator())
ax_res_x.yaxis.set_minor_locator(ticker.AutoMinorLocator())
ax_res.xaxis.set_tick_params(labelsize=17)
ax_res.yaxis.set_tick_params(labelsize=17)

# fig.subplots_adjust(hspace=0.1)
plt.show()

In [ ]:
batman_residuals = flux_data - batman_model

In [ ]:
"""
Batman model only w/ flux data
residuals
"""


#Panel settings
left = 0.1
bottom = 0.1
width = 0.65
height = 0.9
spacing = -1.15

rect = [left, bottom, width, height]
rect_res = [left, bottom+height+spacing, width, 0.15]

fig = plt.figure(figsize=(16, 6), dpi=500)
ax = fig.add_axes(rect)

ax_res = fig.add_axes(rect_res, sharex=ax)

#Top panel
ax.errorbar(t, flux_data, yerr=yerrs,fmt='.', markersize=5, color='black', ecolor='LightGrey', label="Data", zorder=0)
ax.plot(t_range, batman_model_extend_range, color='red',label="Model", lw=2, zorder=10)

ax.set(ylabel="Normalised flux")
ax.set_ylim([0.986, 1.01])
ax.xaxis.label.set_size(17)
ax.yaxis.label.set_size(17)

# Mirror y
axy = ax.twinx()
axy.set_ylim(ax.get_ylim())
axy.get_yaxis().set_ticklabels([])
axy.tick_params(which="both", direction="in")#, labelright=True, labelleft=True)

# Mirror x
axx = ax.twiny()
axx.set_xlim(ax.get_xlim())
axx.get_xaxis().set_ticklabels([])
axx.tick_params(which="both", direction="in")


ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
axx.xaxis.set_minor_locator(ticker.AutoMinorLocator())
axy.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax.xaxis.set_tick_params(labelsize=17)
ax.yaxis.set_tick_params(labelsize=17)

"""
Residuals panel
"""
ax_res.scatter(t, batman_residuals, color='black', s=5)
# ax_res.set_ylim((-max(abs(batman_residuals))*1.2, max(abs(batman_residuals)*1.2)))

ax_res.set(xlabel="Phase", ylabel="Residuals")
ax_res.xaxis.label.set_size(17)
ax_res.yaxis.label.set_size(17)

# Mirror y
ax_res_y = ax_res.twinx()
ax_res_y.set_ylim(ax_res.get_ylim())
ax_res_y.get_yaxis().set_ticklabels([])
ax_res_y.tick_params(which="both", direction="in")

# Mirror x
ax_res_x = ax_res.twiny()
ax_res_x.set_xlim(ax_res.get_xlim())
ax_res_x.get_xaxis().set_ticklabels([])
ax_res_x.tick_params(which="both", direction="in")

ax_res.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax_res.yaxis.set_minor_locator(ticker.AutoMinorLocator())
ax_res_x.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax_res_y.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax_res.xaxis.set_tick_params(labelsize=17)
ax_res.yaxis.set_tick_params(labelsize=17)

# fig.subplots_adjust(hspace=0.1)
plt.show()

In [ ]:
"""
batman + gp model w/ flux data 
residuals
"""

#Panel settings
left = 0.1
bottom = 0.1
width = 0.65
height = 0.9
spacing = -1.15

rect = [left, bottom, width, height]
rect_res = [left, bottom+height+spacing, width, 0.15]

fig = plt.figure(figsize=(16, 6), dpi=500)
ax = fig.add_axes(rect)

ax_res = fig.add_axes(rect_res, sharex=ax)

#Top panel
ax.errorbar(t, flux_data, yerr=yerrs,fmt='.', markersize=5, color='black', ecolor='LightGrey', label="Data", zorder=0)
ax.plot(t, model_flux, color='red',label="Model", lw=2, zorder=10)

ax.set(ylabel="Normalised flux")
ax.set_ylim([0.986, 1.01])
ax.xaxis.label.set_size(17)
ax.yaxis.label.set_size(17)

# Mirror y
axy = ax.twinx()
axy.set_ylim(ax.get_ylim())
axy.get_yaxis().set_ticklabels([])
axy.tick_params(which="both", direction="in")#, labelright=True, labelleft=True)

# Mirror x
axx = ax.twiny()
axx.set_xlim(ax.get_xlim())
axx.get_xaxis().set_ticklabels([])
axx.tick_params(which="both", direction="in")


ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
axx.xaxis.set_minor_locator(ticker.AutoMinorLocator())
axy.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax.xaxis.set_tick_params(labelsize=17)
ax.yaxis.set_tick_params(labelsize=17)

"""
Residuals panel
"""
ax_res.scatter(t, gp_batman_residuals, color='black', s=5)
# ax_res.set_ylim((-max(abs(gp_batman_residuals))*1.2, max(abs(gp_batman_residuals)*1.2)))

ax_res.set(xlabel="Phase", ylabel="Residuals")
ax_res.xaxis.label.set_size(17)
ax_res.yaxis.label.set_size(17)

# Mirror y
ax_res_y = ax_res.twinx()
ax_res_y.set_ylim(ax_res.get_ylim())
ax_res_y.get_yaxis().set_ticklabels([])
ax_res_y.tick_params(which="both", direction="in")

# Mirror x
ax_res_x = ax_res.twiny()
ax_res_x.set_xlim(ax_res.get_xlim())
ax_res_x.get_xaxis().set_ticklabels([])
ax_res_x.tick_params(which="both", direction="in")

ax_res.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax_res.yaxis.set_minor_locator(ticker.AutoMinorLocator())
ax_res_x.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax_res_y.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax_res.xaxis.set_tick_params(labelsize=17)
ax_res.yaxis.set_tick_params(labelsize=17)

# fig.subplots_adjust(hspace=0.1)
plt.show()

In [ ]:
detrended_batman_residuals = detrended_flux_data - batman_model

In [ ]:
"""
batman model w/ detrended flux
residuals
"""

#Panel settings
left = 0.1
bottom = 0.1
width = 0.65
height = 0.9
spacing = -1.15

rect = [left, bottom, width, height]
rect_res = [left, bottom+height+spacing, width, 0.15]

fig = plt.figure(figsize=(16, 6), dpi=500)
ax = fig.add_axes(rect)

ax_res = fig.add_axes(rect_res, sharex=ax)

#Top panel
ax.errorbar(t, detrended_flux_data, yerr=yerrs,fmt='.', markersize=5, color='black', ecolor='LightGrey', label="Data", zorder=0)
ax.plot(t, batman_model, color='red',label="Model", lw=2, zorder=10)

ax.set(ylabel="Normalised flux")
ax.set_ylim([0.986, 1.01])
ax.xaxis.label.set_size(17)
ax.yaxis.label.set_size(17)

# Mirror y
axy = ax.twinx()
axy.set_ylim(ax.get_ylim())
axy.get_yaxis().set_ticklabels([])
axy.tick_params(which="both", direction="in")#, labelright=True, labelleft=True)

# Mirror x
axx = ax.twiny()
axx.set_xlim(ax.get_xlim())
axx.get_xaxis().set_ticklabels([])
axx.tick_params(which="both", direction="in")


ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
axx.xaxis.set_minor_locator(ticker.AutoMinorLocator())
axy.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax.xaxis.set_tick_params(labelsize=17)
ax.yaxis.set_tick_params(labelsize=17)

"""
Residuals panel
"""
ax_res.scatter(t, detrended_batman_residuals, color='black', s=5)
# ax_res.set_ylim((-max(abs(detrended_batman_residuals))*1.2, max(abs(detrended_batman_residuals)*1.2)))

ax_res.set(xlabel="Phase", ylabel="Residuals")
ax_res.xaxis.label.set_size(17)
ax_res.yaxis.label.set_size(17)

# Mirror y
ax_res_y = ax_res.twinx()
ax_res_y.set_ylim(ax_res.get_ylim())
ax_res_y.get_yaxis().set_ticklabels([])
ax_res_y.tick_params(which="both", direction="in")

# Mirror x
ax_res_x = ax_res.twiny()
ax_res_x.set_xlim(ax_res.get_xlim())
ax_res_x.get_xaxis().set_ticklabels([])
ax_res_x.tick_params(which="both", direction="in")

ax_res.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax_res.yaxis.set_minor_locator(ticker.AutoMinorLocator())
ax_res_x.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax_res_y.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax_res.xaxis.set_tick_params(labelsize=17)
ax_res.yaxis.set_tick_params(labelsize=17)

# fig.subplots_adjust(hspace=0.1)
plt.show()